In [15]:
import pandas as pd
import numpy as np
import pyodbc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

In [16]:
servername = 'SIU1\\SQLEXPRESS'

In [17]:
conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=UnitedOutdoors; Trusted_Connection=yes')

In [18]:
orders = pd.read_sql_query('SELECT * FROM order_details', conn)
orders

C:\Users\tinep\AppData\Local\Temp\ipykernel_21252\1551285142.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = pd.read_sql_query('SELECT * FROM order_details', conn)


,order_details_sk,order_id,product_quantity,product_listprice,order_date,shipping_date,product_productid,product_sk,employee_employeeid,employee_sk,customer_id,customer_sk,source
0,1,2001,12,9.00,1996-03-14,1996-09-15,300,582,299.0,314.0,101,92,AC
1,2,2001,12,14.00,1996-03-14,1996-09-14,301,583,299.0,314.0,101,92,AC
2,3,2001,12,14.00,1996-03-14,1996-09-14,302,584,299.0,314.0,101,92,AC
3,4,2002,24,9.00,1996-03-18,1996-09-18,400,585,467.0,321.0,102,93,AC
4,5,2002,24,10.00,1996-03-18,1996-09-18,401,586,467.0,321.0,102,93,AC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124570,124571,75122,1,21.98,2014-06-30,2014-07-07,878,383,NaN,NaN,15868,5798,AW
124571,124572,75122,1,8.99,2014-06-30,2014-07-07,712,217,NaN,NaN,15868,5798,AW
124572,124573,75123,1,21.98,2014-06-30,2014-07-07,878,383,NaN,NaN,18759,8693,AW
124573,124574,75123,1,159.00,2014-06-30,2014-07-07,879,384,NaN,NaN,18759,8693,AW


In [24]:
df = orders.loc[:, ['product_quantity', 'product_listprice', 'shipping_date', 'order_date', 'product_productid', 'customer_id']]
df

,product_quantity,product_listprice,shipping_date,order_date,product_productid,customer_id
0,12,9.00,1996-09-15,1996-03-14,300,101
1,12,14.00,1996-09-14,1996-03-14,301,101
2,12,14.00,1996-09-14,1996-03-14,302,101
3,24,9.00,1996-09-18,1996-03-18,400,102
4,24,10.00,1996-09-18,1996-03-18,401,102
...,...,...,...,...,...,...
124570,1,21.98,2014-07-07,2014-06-30,878,15868
124571,1,8.99,2014-07-07,2014-06-30,712,15868
124572,1,21.98,2014-07-07,2014-06-30,878,18759
124573,1,159.00,2014-07-07,2014-06-30,879,18759


In [25]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['order_month'] = df['order_date'].dt.month
dummies_order_month = pd.get_dummies(df['order_month'], prefix='order_month')
dummies_order_month

,order_month_1,order_month_2,order_month_3,order_month_4,order_month_5,order_month_6,order_month_7,order_month_8,order_month_9,order_month_10,order_month_11,order_month_12
0,False,False,True,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
124570,False,False,False,False,False,True,False,False,False,False,False,False
124571,False,False,False,False,False,True,False,False,False,False,False,False
124572,False,False,False,False,False,True,False,False,False,False,False,False
124573,False,False,False,False,False,True,False,False,False,False,False,False


In [29]:
features = ['product_quantity', 'product_listprice', 'shipping_date', 'order_date', 'product_productid', 'customer_id']


In [30]:
X = pd.concat([dummies_order_month, df[features]], axis=1)
y = df['product_productid']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)